<a href="https://colab.research.google.com/github/mjiii25/prac_class/blob/main/LGBM%20%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [105]:
train = pd.read_csv('/content/drive/MyDrive/energy/new_train.csv')
test = pd.read_csv('/content/drive/MyDrive/energy/new_test.csv')
submission = pd.read_csv('/content/drive/MyDrive/energy/sample_submission.csv')

In [106]:
train.head()

,num,date_time,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,date,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,0,1,6,2020-06-01,0,0,63.43008,0,0,0,0,0,0.0,-8.4,18.973374,1
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,1,1,6,2020-06-01,0,0,63.56993,0,0,0,0,0,0.0,-16.7,18.960338,1
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2,1,6,2020-06-01,0,0,63.22775,0,0,0,0,0,0.0,-25.2,18.658598,1
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,3,1,6,2020-06-01,0,0,62.54339,0,0,0,0,0,0.0,-34.1,18.218957,1
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,4,1,6,2020-06-01,0,0,62.39760,0,0,0,0,0,0.0,-43.1,18.081523,1


In [107]:
train.shape

(122400, 26)

In [108]:
test.head()

,num,date_time,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,date,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
0,1,2020-08-25 00:00:00,27.800000,1.500000,74.000000,0.0,0.0,0,0,0,25,8,2020-08-25,1,0,78.602280,0,1,0,0,0,0.0,1.800000,30.027122,2
1,1,2020-08-25 01:00:00,27.633333,1.366667,75.333333,0.0,0.0,0,0,1,25,8,2020-08-25,1,0,78.519273,0,1,0,0,0,0.0,3.433333,29.859547,2
2,1,2020-08-25 02:00:00,27.466667,1.233333,76.666667,0.0,0.0,0,0,2,25,8,2020-08-25,1,0,78.431867,0,1,0,0,0,0.0,4.900000,29.694705,2
3,1,2020-08-25 03:00:00,27.300000,1.100000,78.000000,0.0,0.0,0,0,3,25,8,2020-08-25,1,0,78.340060,0,1,0,0,0,0.0,6.200000,29.533017,2
4,1,2020-08-25 04:00:00,26.900000,1.166667,79.666667,0.0,0.0,0,0,4,25,8,2020-08-25,1,0,77.912697,0,1,0,0,0,0.0,7.100000,29.116617,2


In [109]:
test.shape

(10080, 25)

## 군집화 결과 (수연님)

In [138]:
# 1번 군집 
list1 = [1, 31, 30, 60, 3, 5, 24,32, 33, 15]
cl1 = train.drop(['date_time', 'date'], axis = 1)
cl1 = cl1[cl1['num'].isin(list1)==True]
#cl1 = train[train['num'].isin(list1)==True]
print(train.shape, cl1.shape)
print('1번 군집은 전체 데이터의 {0:.2f}'.format(cl1.shape[0]/train.shape[0] *100 )+' %를 차지함')

(122400, 26) (20400, 24)
1번 군집은 전체 데이터의 16.67 %를 차지함


In [139]:
# 2번 군집 
list1 = [2, 6, 7, 8, 13, 14, 16, 17, 18, 22, 23, 25, 26, 27, 37, 46, 47, 48, 52, 53, 54, 55, 56, 57 ,35, 38, 43, 44, 39, 45]
cl2 = train.drop(['date_time', 'date'], axis = 1)
cl2 = cl2[cl2['num'].isin(list1)==True]
#cl2 = train[train['num'].isin(list1)==True]
print(train.shape, cl2.shape)
print('2번 군집은 전체 데이터의 {0:.2f}'.format(cl2.shape[0]/train.shape[0] *100 )+' %를 차지함')

(122400, 26) (61200, 24)
2번 군집은 전체 데이터의 50.00 %를 차지함


In [141]:
# 3번 군집
l2 = [19,20,21,29,36,40,41,49,50,58,59]
cl3 = train.drop(['date_time', 'date'], axis = 1)
cl3 = cl3[cl3['num'].isin(l2)==True]
#cl3 = train[train['num'].isin(l2)==True]
print(train.shape, cl3.shape)
print('3번 군집은 전체 데이터의 {0:.2f}'.format(cl3.shape[0]/train.shape[0] *100 )+' %를 차지함')

(122400, 26) (22440, 24)
3번 군집은 전체 데이터의 18.33 %를 차지함


In [142]:
# 4번 군집
l3 = [14,9,10,11,12,30,42]
cl4 = train.drop(['date_time', 'date'], axis = 1)
cl4 = cl4[cl4['num'].isin(l3)==True]
#cl4 = train[train['num'].isin(l3)==True]
print(train.shape, cl4.shape)
print('4번 군집은 전체 데이터의 {0:.2f}'.format(cl4.shape[0]/train.shape[0] *100 )+' %를 차지함')

(122400, 26) (14280, 24)
4번 군집은 전체 데이터의 11.67 %를 차지함


In [143]:
# 5번 군집
l4 = [5,21,28,32,51, 34]
cl5 = train.drop(['date_time', 'date'], axis = 1)
cl5 = cl5[cl5['num'].isin(l4)==True]
#cl5 = train[train['num'].isin(l4)==True]
print(train.shape, cl5.shape)
print('5번 군집은 전체 데이터의 {0:.2f}'.format(cl5.shape[0]/train.shape[0] *100 )+' %를 차지함')

(122400, 26) (12240, 24)
5번 군집은 전체 데이터의 10.00 %를 차지함


In [144]:
# 군집 5개 완성!
cl1.reset_index(inplace=True, drop=True)
cl2.reset_index(inplace=True, drop=True)
cl3.reset_index(inplace=True, drop=True)
cl4.reset_index(inplace=True, drop=True)
cl5.reset_index(inplace=True, drop=True)

In [145]:
# 기술통계량 확인
train.describe()

,num,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000
mean,30.500000,2324.830866,24.251713,2.151641,80.169848,0.514989,0.213533,0.683333,0.483333,11.500000,14.835294,6.929412,2.964706,0.282353,73.461061,0.352990,0.320180,0.066953,0.011797,0.001029,0.418172,-21.071605,26.362236,1.105882
std,17.318173,2058.999326,3.407902,1.514475,15.525862,2.624505,0.370517,0.465178,0.499724,6.922215,8.446192,0.793925,2.014351,0.450146,4.778330,0.477902,0.466547,0.249941,0.107974,0.032068,1.377256,34.111471,3.542213,0.307688
min,1.000000,0.000000,11.100000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,0.000000,52.344210,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-151.300000,13.436363,1.000000
25%,15.750000,1055.268000,21.800000,1.100000,70.000000,0.000000,0.000000,0.000000,0.000000,5.750000,8.000000,6.000000,1.000000,0.000000,70.035960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-45.300000,23.833377,1.000000
50%,30.500000,1700.352000,24.200000,1.900000,84.000000,0.000000,0.000000,1.000000,0.000000,11.500000,15.000000,7.000000,3.000000,0.000000,73.615850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-20.800000,26.278986,1.000000
75%,45.250000,2780.487000,26.500000,2.900000,93.000000,0.000000,0.300000,1.000000,1.000000,17.250000,22.000000,8.000000,5.000000,1.000000,77.074510,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.800000,28.675256,1.000000
max,60.000000,17739.225000,36.300000,20.100000,100.000000,81.500000,1.000000,1.000000,1.000000,23.000000,31.000000,8.000000,6.000000,1.000000,86.661790,1.000000,1.000000,1.000000,1.000000,1.000000,20.000000,99.200000,38.971260,2.000000


In [119]:
cl1.describe()

,num,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,20400.00000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.00000,20400.00000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000
mean,23.40000,4000.199927,24.178363,2.492598,82.016333,0.507814,0.218363,0.80000,0.40000,11.500000,14.835294,6.929412,2.964706,0.282353,73.546570,0.345833,0.324167,0.072206,0.011863,0.000294,0.436984,-21.952456,26.205680,1.105882
std,17.13051,2057.588422,3.314664,1.844740,15.074272,2.655440,0.375753,0.40001,0.48991,6.922356,8.446365,0.793941,2.014392,0.450155,4.776976,0.475651,0.468074,0.258835,0.108271,0.017148,1.482577,33.189183,3.495996,0.307695
min,1.00000,884.304000,13.200000,0.000000,19.000000,0.000000,0.000000,0.00000,0.00000,0.000000,1.000000,6.000000,0.000000,0.000000,55.796960,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-137.400000,15.177703,1.000000
25%,5.00000,2504.682000,21.700000,1.300000,72.000000,0.000000,0.000000,1.00000,0.00000,5.750000,8.000000,6.000000,1.000000,0.000000,70.054380,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-46.000000,23.637000,1.000000
50%,27.00000,3204.576000,24.100000,2.100000,86.000000,0.000000,0.000000,1.00000,0.00000,11.500000,15.000000,7.000000,3.000000,0.000000,73.709460,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-20.900000,26.192815,1.000000
75%,32.00000,5593.320000,26.500000,3.200000,95.000000,0.000000,0.300000,1.00000,1.00000,17.250000,22.000000,8.000000,5.000000,1.000000,77.176620,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,28.597333,1.000000
max,60.00000,9802.080000,35.200000,20.100000,100.000000,70.400000,1.000000,1.00000,1.00000,23.000000,31.000000,8.000000,6.000000,1.000000,86.569350,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000,76.000000,38.157722,2.000000


In [120]:
cl2.describe()

,num,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.00000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000,61200.000000
mean,32.433333,2314.197573,24.368634,1.894237,79.907824,0.503023,0.207758,0.766667,0.533333,11.500000,14.835294,6.929412,2.964706,0.282353,73.610545,0.34866,0.330327,0.068186,0.013072,0.001520,0.411746,-19.671977,26.541799,1.105882
std,17.038690,2223.567823,3.428643,1.305899,15.659162,2.542276,0.365079,0.422956,0.498892,6.922243,8.446227,0.793928,2.014359,0.450148,4.771457,0.47655,0.470335,0.252067,0.113584,0.038953,1.330571,34.028977,3.523607,0.307689
min,2.000000,0.000000,11.700000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,0.000000,53.397080,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-150.500000,13.975392,1.000000
25%,17.000000,1098.684000,21.900000,0.900000,69.000000,0.000000,0.000000,1.000000,0.000000,5.750000,8.000000,6.000000,1.000000,0.000000,70.193480,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-43.400000,24.071890,1.000000
50%,36.000000,1656.288000,24.300000,1.700000,84.000000,0.000000,0.000000,1.000000,1.000000,11.500000,15.000000,7.000000,3.000000,0.000000,73.835330,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-19.100000,26.454967,1.000000
75%,47.000000,2561.328000,26.700000,2.600000,93.000000,0.000000,0.300000,1.000000,1.000000,17.250000,22.000000,8.000000,5.000000,1.000000,77.181540,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,3.900000,28.823881,1.000000
max,57.000000,17739.225000,35.700000,12.800000,100.000000,81.500000,1.000000,1.000000,1.000000,23.000000,31.000000,8.000000,6.000000,1.000000,86.661790,1.00000,1.000000,1.000000,1.000000,1.000000,20.000000,99.200000,38.872384,2.000000


In [121]:
cl3.describe()

,num,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000,22440.000000
mean,38.363636,1270.805991,24.194519,2.222576,80.274866,0.531003,0.215989,0.363636,0.272727,11.500000,14.835294,6.929412,2.964706,0.282353,73.388815,0.357308,0.310918,0.069474,0.011497,0.000624,0.222029,-21.755918,26.302397,1.105882
std,14.066861,719.891681,3.422644,1.464918,15.291852,2.653663,0.372133,0.481056,0.445372,6.922341,8.446346,0.793939,2.014388,0.450154,4.827520,0.479217,0.462880,0.254264,0.106610,0.024970,0.983552,34.445252,3.545642,0.307694
min,19.000000,275.238000,11.100000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,0.000000,52.344210,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-151.300000,13.436363,1.000000
25%,21.000000,856.170000,21.700000,1.200000,70.000000,0.000000,0.000000,0.000000,0.000000,5.750000,8.000000,6.000000,1.000000,0.000000,69.982168,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-46.400000,23.779269,1.000000
50%,40.000000,1097.941500,24.100000,2.000000,84.000000,0.000000,0.000000,0.000000,0.000000,11.500000,15.000000,7.000000,3.000000,0.000000,73.499380,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-22.100000,26.161911,1.000000
75%,50.000000,1477.251000,26.500000,3.000000,93.000000,0.000000,0.300000,1.000000,1.000000,17.250000,22.000000,8.000000,5.000000,1.000000,77.084440,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.200000,28.641924,1.000000
max,59.000000,4905.684000,35.500000,12.800000,100.000000,70.400000,1.000000,1.000000,1.000000,23.000000,31.000000,8.000000,6.000000,1.000000,86.569350,1.000000,1.000000,1.000000,1.000000,1.000000,13.898305,86.500000,38.157722,2.000000


In [122]:
cl4.describe()

,num,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000,14280.000000
mean,18.285714,2568.672497,23.870819,2.983543,82.697787,0.470763,0.235112,0.714286,0.714286,11.500000,14.835294,6.929412,2.964706,0.282353,73.148073,0.360924,0.257983,0.082143,0.016106,0.001961,0.778144,-25.631632,25.827033,1.105882
std,11.744276,1955.357567,3.381474,2.173993,14.189761,2.665762,0.389126,0.451770,0.451770,6.922429,8.446453,0.793949,2.014414,0.450160,4.976581,0.480285,0.437540,0.274592,0.125889,0.044239,1.890730,35.929885,3.602703,0.307698
min,9.000000,347.652000,12.600000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,0.000000,54.935640,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-135.200000,15.177703,1.000000
25%,10.000000,1183.437000,21.400000,1.500000,74.000000,0.000000,0.000000,0.000000,0.000000,5.750000,8.000000,6.000000,1.000000,0.000000,69.593430,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-51.400000,23.200383,1.000000
50%,12.000000,2030.022000,23.600000,2.500000,86.000000,0.000000,0.000000,1.000000,1.000000,11.500000,15.000000,7.000000,3.000000,0.000000,72.911360,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-29.100000,25.612251,1.000000
75%,30.000000,3574.044000,26.100000,3.900000,94.000000,0.000000,0.400000,1.000000,1.000000,17.250000,22.000000,8.000000,5.000000,1.000000,76.863530,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,-0.100000,28.197786,1.000000
max,42.000000,9802.080000,35.700000,20.100000,100.000000,70.400000,1.000000,1.000000,1.000000,23.000000,31.000000,8.000000,6.000000,1.000000,86.661790,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000,99.200000,38.872384,2.000000


In [123]:
cl5.describe()

,num,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
count,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.00000,12240.00000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.000000,12240.0,12240.000000,12240.000000,12240.000000,12240.000000
mean,28.500000,1941.539742,24.218946,2.561887,78.801552,0.577116,0.208489,0.50000,0.50000,11.500000,14.835294,6.929412,2.964706,0.282353,73.313394,0.364297,0.326879,0.058252,0.004984,0.0,0.464551,-21.463096,26.208722,1.105882
std,13.890012,722.919915,3.261309,1.531287,15.274456,2.980738,0.366618,0.50002,0.50002,6.922469,8.446503,0.793954,2.014425,0.450163,4.594105,0.481252,0.469092,0.234228,0.070422,0.0,1.525493,32.690524,3.468719,0.307700
min,5.000000,525.204000,13.200000,0.000000,19.000000,0.000000,0.000000,0.00000,0.00000,0.000000,1.000000,6.000000,0.000000,0.000000,55.796960,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-137.400000,15.727177,1.000000
25%,21.000000,1416.028500,21.800000,1.600000,69.000000,0.000000,0.000000,0.00000,0.00000,5.750000,8.000000,6.000000,1.000000,0.000000,70.040500,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-45.100000,23.687612,1.000000
50%,30.000000,2032.695000,24.200000,2.300000,82.000000,0.000000,0.000000,0.50000,0.50000,11.500000,15.000000,7.000000,3.000000,0.000000,73.464880,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-20.700000,26.142099,1.000000
75%,34.000000,2422.224000,26.400000,3.200000,91.000000,0.000000,0.300000,1.00000,1.00000,17.250000,22.000000,8.000000,5.000000,1.000000,76.904520,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,1.800000,28.503867,1.000000
max,51.000000,3890.160000,35.200000,12.800000,100.000000,70.400000,1.000000,1.00000,1.00000,23.000000,31.000000,8.000000,6.000000,1.000000,84.754130,1.000000,1.000000,1.000000,1.000000,0.0,13.750000,76.000000,38.157722,2.000000


## LGBM 모델링

[참고] https://www.kaggle.com/janged/xgb-lgb

In [146]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [147]:
def print_best_params(model, params, cl_x, cl_y):
    grid_model = GridSearchCV(
        model, 
        param_grid = params,
        cv=5,
        scoring='neg_mean_squared_error')

    grid_model.fit(cl_x, cl_y)
    rmse = np.sqrt(-1*grid_model.best_score_)
    print(
        '{0} 5 CV 시 최적 평균 RMSE 값 {1}, 최적 alpha:{2}'.format(model.__class__.__name__, np.round(rmse, 4), grid_model.best_params_))
    return grid_model.best_estimator_

In [131]:
SEED = 30

In [149]:
lgb_params = {
    'objective':['regression'],
    'num_leave' : [1],
    'learning_rate' : [0.05],
    'n_estimators':[1000],
    'max_bin' : [80],
    'gpu_id':[0] ,         
    'tree_method':['gpu_hist'],
    'refit':[True]
}

lgb_model = LGBMRegressor()

#### cluster1

In [148]:
cl1_x = cl1.drop('target', axis = 1)
cl1_y = cl1[['target']]

In [132]:
x_train, x_test, y_train, y_test = train_test_split(cl1_x, cl1_y, random_state=SEED)

In [150]:
lgb_estimator_cl1 = print_best_params(lgb_model, lgb_params, cl1_x, cl1_y)

LGBMRegressor 5 CV 시 최적 평균 RMSE 값 2472.0638, 최적 alpha:{'gpu_id': 0, 'learning_rate': 0.05, 'max_bin': 80, 'n_estimators': 1000, 'num_leave': 1, 'objective': 'regression', 'refit': True, 'tree_method': 'gpu_hist'}


#### cluster 2

In [151]:
cl2_x = cl2.drop('target', axis = 1)
cl2_y = cl2[['target']]

In [152]:
x_train, x_test, y_train, y_test = train_test_split(cl2_x, cl2_y, random_state=SEED)

In [172]:
lgb_estimator_cl2 = print_best_params(lgb_model, lgb_params, cl2_x, cl2_y)

LGBMRegressor 5 CV 시 최적 평균 RMSE 값 2278.689, 최적 alpha:{'gpu_id': 0, 'learning_rate': 0.05, 'max_bin': 80, 'n_estimators': 1000, 'num_leave': 1, 'objective': 'regression', 'refit': True, 'tree_method': 'gpu_hist'}


#### cluster 3

In [154]:
cl3_x = cl3.drop('target', axis = 1)
cl3_y = cl3[['target']]

In [155]:
x_train, x_test, y_train, y_test = train_test_split(cl3_x, cl3_y, random_state=SEED)

In [156]:
lgb_estimator_cl3 = print_best_params(lgb_model, lgb_params, cl3_x, cl3_y)

LGBMRegressor 5 CV 시 최적 평균 RMSE 값 646.9296, 최적 alpha:{'gpu_id': 0, 'learning_rate': 0.05, 'max_bin': 80, 'n_estimators': 1000, 'num_leave': 1, 'objective': 'regression', 'refit': True, 'tree_method': 'gpu_hist'}


#### cluster 4

In [157]:
cl4_x = cl4.drop('target', axis = 1)
cl4_y = cl4[['target']]

In [158]:
x_train, x_test, y_train, y_test = train_test_split(cl4_x, cl4_y, random_state=SEED)

In [159]:
lgb_estimator_cl4 = print_best_params(lgb_model, lgb_params, cl4_x, cl4_y)

LGBMRegressor 5 CV 시 최적 평균 RMSE 값 1278.4376, 최적 alpha:{'gpu_id': 0, 'learning_rate': 0.05, 'max_bin': 80, 'n_estimators': 1000, 'num_leave': 1, 'objective': 'regression', 'refit': True, 'tree_method': 'gpu_hist'}


#### cluster 5

In [160]:
cl5_x = cl5.drop('target', axis = 1)
cl5_y = cl5[['target']]

In [161]:
x_train, x_test, y_train, y_test = train_test_split(cl5_x, cl5_y, random_state=SEED)

In [162]:
lgb_estimator_cl5 = print_best_params(lgb_model, lgb_params, cl5_x, cl5_y)

LGBMRegressor 5 CV 시 최적 평균 RMSE 값 539.688, 최적 alpha:{'gpu_id': 0, 'learning_rate': 0.05, 'max_bin': 80, 'n_estimators': 1000, 'num_leave': 1, 'objective': 'regression', 'refit': True, 'tree_method': 'gpu_hist'}


## predict

In [164]:
testt = test.drop(['date_time', 'date'], axis = 1)

In [166]:
testt.head()

,num,temperature,windspeed,humidity,precipitation,insolation,nelec_cool,solar,hour,day,month,weekday,weekend,DI,DI_70,DI_75,DI_80,DI_83,DI_86,SL,CDH,TW,corona
0,1,27.800000,1.500000,74.000000,0.0,0.0,0,0,0,25,8,1,0,78.602280,0,1,0,0,0,0.0,1.800000,30.027122,2
1,1,27.633333,1.366667,75.333333,0.0,0.0,0,0,1,25,8,1,0,78.519273,0,1,0,0,0,0.0,3.433333,29.859547,2
2,1,27.466667,1.233333,76.666667,0.0,0.0,0,0,2,25,8,1,0,78.431867,0,1,0,0,0,0.0,4.900000,29.694705,2
3,1,27.300000,1.100000,78.000000,0.0,0.0,0,0,3,25,8,1,0,78.340060,0,1,0,0,0,0.0,6.200000,29.533017,2
4,1,26.900000,1.166667,79.666667,0.0,0.0,0,0,4,25,8,1,0,77.912697,0,1,0,0,0,0.0,7.100000,29.116617,2


#### cluster 1

In [175]:
lgb_preds_cl1 = lgb_estimator_cl1.predict(testt)

In [176]:
preds_cl1 = 0.5*lgb_preds_cl1
preds_cl1

array([377.78603392, 351.67493721, 352.78248973, ..., 954.48645093,
       885.97281628, 842.08241467])

#### cluster 2

In [177]:
lgb_preds_cl2 = lgb_estimator_cl2.predict(testt)

In [178]:
preds_cl2 = 0.5*lgb_preds_cl2
preds_cl2

array([377.78603392, 351.67493721, 352.78248973, ..., 954.48645093,
       885.97281628, 842.08241467])

#### cluster3

In [179]:
lgb_preds_cl3 = lgb_estimator_cl3.predict(testt)

In [180]:
preds_cl3 = 0.5*lgb_preds_cl3
preds_cl3

array([454.34754254, 366.36549018, 351.81829585, ..., 510.62492227,
       513.61768141, 819.40236605])

#### cluster4

In [181]:
lgb_preds_cl4 = lgb_estimator_cl4.predict(testt)

In [182]:
preds_cl4 = 0.5*lgb_preds_cl4
preds_cl4

array([ 612.59154573,  592.12346895,  592.12072624, ...,  945.53929067,
        774.30641182, 1493.69873856])

#### cluster5

In [183]:
lgb_preds_cl5 = lgb_estimator_cl5.predict(testt)

In [184]:
preds_cl5 = 0.5*lgb_preds_cl5
preds_cl5

array([1467.30206054, 1403.8577935 , 1392.44273292, ...,  651.04921355,
        559.36160691,  437.14150742])

--------------------------------------------------------------------------------------------------------------



DACON 베이스라인

In [ ]:
cross = KFold(n_splits = 5, shuffle = True, random_state = 42)
folds = []
for train_idx, valid_idx in cross.split(cl1_x, cl1_y):
  folds.append((train_idx, valid_idx))

In [ ]:
models={}
for fold in range(5):
    print(f'===================={fold+1}=======================')
    train_idx, valid_idx=folds[fold]
    X_train=cl1_x.iloc[train_idx, :]
    y_train=cl1_y.iloc[train_idx, :]
    X_valid=cl1_x.iloc[valid_idx, :]
    y_valid=cl1_y.iloc[valid_idx, :]
    
    model=LGBMRegressor(n_estimators=100)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], 
             early_stopping_rounds=30, verbose=100)
    models[fold]=model
    
    print(f'================================================\n\n')

====================1=======================


ValueError: ignored